In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok

# Replace with your actual token
ngrok.set_auth_token("2x21WASjly9m59LrRHFQKnVsCNH_8641niycL8rbRnHMs6MSH")

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import uvicorn

app = FastAPI()

class QuestionInput(BaseModel):
    question: str

@app.post("/classify-question")
def classify(input_data: QuestionInput):
    question = input_data.question.strip().lower()

    if not question:
        raise HTTPException(status_code=400, detail="Empty question.")

    # Rule-based keywords
    topics = {
        "Math": ["theorem", "algebra", "geometry", "triangle"],
        "Science": ["atom", "gravity", "photosynthesis", "chemical"],
        "English": ["noun", "verb", "poem", "grammar"]
    }

    scores = {topic: sum(1 for kw in kws if kw in question) for topic, kws in topics.items()}
    best_topic = max(scores, key=scores.get)
    total = sum(scores.values())
    confidence = round(scores[best_topic] / total, 2) if total > 0 else 0.0

    result = {
        "topic": best_topic if confidence > 0 else "Unknown",
        "confidence": confidence
    }
    return result

In [ ]:
# Run only once to patch asyncio event loop
nest_asyncio.apply()

# Start FastAPI with Uvicorn
from pyngrok import ngrok

# Create a tunnel to the uvicorn port
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Start the server
uvicorn.run(app, host="0.0.0.0", port=8000)


Public URL: NgrokTunnel: "https://2831-104-196-61-22.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [220]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2406:b400:35:cdc5:c58b:5041:8fbe:a2cc:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2406:b400:35:cdc5:c58b:5041:8fbe:a2cc:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2406:b400:35:cdc5:c58b:5041:8fbe:a2cc:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2406:b400:35:cdc5:c58b:5041:8fbe:a2cc:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2406:b400:35:cdc5:c58b:5041:8fbe:a2cc:0 - "POST /classify-question HTTP/1.1" 200 OK


In [ ]:
from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel
from typing import Literal
import json
import os

app = FastAPI()

# Input validation
class QuestionInput(BaseModel):
    question: str

# Classification logic (very simple rule-based example)
def classify_question(question: str):
    question_lower = question.lower()
    if any(word in question_lower for word in ["math", "pythagorean", "algebra", "geometry", "calculus", "equation"]):
        return {"topic": "Math", "confidence": 1.0}
    elif any(word in question_lower for word in ["biology", "physics", "gravity", "science", "photosynthesis", "experiment"]):
        return {"topic": "Science", "confidence": 1.0}
    elif any(word in question_lower for word in ["noun", "verb", "english", "grammar", "poem", "essay", "literature"]):
        return {"topic": "English", "confidence": 1.0}
    else:
        return {"topic": "Unknown", "confidence": 0.0}

# File path
LOG_FILE = "questions.json"

# Save question and result to file
def save_to_file(entry):
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r") as f:
            data = json.load(f)
    else:
        data = []

    data.append(entry)

    with open(LOG_FILE, "w") as f:
        json.dump(data, f, indent=4)

# POST endpoint: classify a question
@app.post("/classify-question")
def classify(input: QuestionInput):
    if not input.question.strip():
        raise HTTPException(status_code=400, detail="Question cannot be empty.")

    result = classify_question(input.question)
    result_data = {
        "question": input.question,
        "topic": result["topic"],
        "confidence": result["confidence"]
    }

    save_to_file(result_data)
    return {"topic": result["topic"], "confidence": result["confidence"]}

# GET endpoint: return past classified questions
@app.get("/questions")
def get_logged_questions():
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r") as f:
            return json.load(f)
    else:
        return []
